# 2D Navier-Stokes equations

Note que las primeras derivadas de una función de dos variables, $x,y$ están dadas por 
$$ \frac{\partial f}{\partial x} = \frac{f(x+h,y)-f(x,y)}{h} = \frac{f_{i+1,j}-f_{i,j}}{h} $$
$$ \frac{\partial f}{\partial y} = \frac{f(x,y+h)-f(x,y)}{h} = \frac{f_{i,j+1}-f_{i,j}}{h} $$
$$ \frac{\partial^2 f}{\partial x^2} = \frac{f(x-h,y)-2f(x,y)+f(x+h,y}{h^2} = \frac{f_{i-1,j}-2f_{i,j}+f_{i+1,j}}{h^2}$$
$$ \frac{\partial^2 f}{\partial y^2} = \frac{f(x,y-h)-2f(x,y)+f(x,y+h}{h^2} = \frac{f_{i,j-1}-2f_{i,j}+f_{i,j-1}}{h^2}$$